In [1]:
'''
Set path to the folder of used python files
'''
import sys
sys.path.insert(0, '/Users/lynnjiang/liuGit/Pybook/Clinic-Analysis/Scripts/Data_Cleaning')
from Abstract import Abstract

In [2]:
import re
import numpy as np
import pandas as pd
class Prescriptions(Abstract):
    def __init__(self):
        super().__init__()

    def dosage_value(self, s):
        try:
            return float(s)
        except ValueError:
            return 0.0

    def read_prescriptions_data(self, user_list, prescriptions_file='PRESCRIPTIONS'):
        columns = ['SUBJECT_ID','DRUG','DOSE_VAL_RX','ENDDATE','STARTDATE']
        top200_drugs = 'temp/PRES_TOP200_DRUGS'
        num_drugs = 20
        digit = re.compile('[^\d-]')

        prescriptions_df = self.read_data(prescriptions_file)
        pres_clean_df = prescriptions_df[columns].dropna(subset=[columns[2]])
        frequent_drugs = self.read_data(top200_drugs)[:num_drugs][columns[1]]
        pres_filter_df = pres_clean_df[(pres_clean_df['SUBJECT_ID'].isin(user_list))\
            & pres_clean_df['DRUG'].isin(frequent_drugs)]
        ## Regularize value in ['DOSE_VAL_RX'] to numeric type, e.g. (300-600)->450
        pres_filter_df.DOSE_VAL_RX = pres_filter_df.DOSE_VAL_RX.apply(lambda x: np.average(list(map(self.dosage_value,digit.sub('',str(x)).split('-')))))

        return pres_filter_df

    def to_attributes(self, pres_df):
        ## Get period for patient taking a medicine in each record (ENDDATE-STARTDATE+1) (/day)
        enddate, startdate, period, ave_value = 'ENDDATE', 'STARTDATE', 'PERIOD', 'AVG_DOSE_VAL_RX'
        pres_df[enddate] = pd.to_datetime(pres_df[enddate])
        pres_df[startdate] = pd.to_datetime(pres_df[startdate])
        pres_df[period] = (pres_df[enddate] - pres_df[startdate]).dt.days + 1

#         ## Get average dosage for each patient taking each kind of drug
#         pres_sum_df = pres_df.groupby(['SUBJECT_ID','DRUG'])['DOSE_VAL_RX','PERIOD'].sum().reset_index()
#         pres_sum_df['AVG_DOSE_VAL_RX'] = round(pres_sum_df['DOSE_VAL_RX']/pres_sum_df['PERIOD'],6)

#         ## Transform dataframe to matrix (USER*DRUG)
#         user_prescription_vector = pd.pivot_table(pres_sum_df, index=['SUBJECT_ID'], columns=['DRUG'],\
#                                                   values=['AVG_DOSE_VAL_RX'])
#         ## Imputation: with 0
#         user_final_presvec= user_prescription_vector.fillna(0)
#         ## TODO: reset_index has some problems, the title is contained as a row
#         user_final_presvec = user_final_presvec.reset_index()
#         user_final_presvec.columns = user_final_presvec.columns.droplevel(-1)

        return pres_df



In [3]:

pp = Prescriptions()
user_list = pp.read_data('PATIENTS')['SUBJECT_ID']
pres_df = pp.to_attributes(pp.read_prescriptions_data(user_list))
# pp.write2file(user_vectors,'prestest_uservectors')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [4]:
pres_df[:20]

,SUBJECT_ID,DRUG,DOSE_VAL_RX,ENDDATE,STARTDATE,PERIOD
3,6,D5W,250.0,2175-06-12,2175-06-11,2.0
4,6,Furosemide,20.0,2175-06-12,2175-06-11,2.0
7,6,D5W,250.0,2175-06-12,2175-06-12,1.0
10,6,D5W,250.0,2175-06-13,2175-06-12,2.0
16,6,D5W,250.0,2175-06-14,2175-06-13,2.0
23,13,D5W,250.0,2167-01-09,2167-01-08,2.0
26,13,Docusate Sodium,100.0,2167-01-09,2167-01-08,2.0
27,13,Insulin,0.0,2167-01-09,2167-01-08,2.0
32,13,Lorazepam,1.0,2167-01-09,2167-01-09,1.0
33,13,Magnesium Sulfate,2.0,2167-01-09,2167-01-09,1.0


In [5]:
pp.read_data('PRESCRIPTIONS')[:20]

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2909: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO
5,2214774,6,107064,NaN,2175-06-11 00:00:00,2175-06-15 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF0,014198,56016975.0,Check with MD for Dose,1,dose,1,dose,PO
6,2215525,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
7,2216266,6,107064,NaN,2175-06-12 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
8,2215526,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
9,2214778,6,107064,NaN,2175-06-12 00:00:00,2175-06-13 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF2,006561,56017075.0,2mg Tab,2,mg,1,TAB,PO


In [19]:
pres_df.groupby(['SUBJECT_ID','DRUG'])['DOSE_VAL_RX'].rolling(window=365, freq='D',min_periods=1).mean

<bound method Rolling.mean of RollingGroupby [window=365,min_periods=1,center=False,axis=0]>